# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Trees').getOrCreate()
spark

In [4]:
df_dog = spark.read.csv('dog_food.csv',header=True,inferSchema=True)
df_dog.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [8]:
#To check the number of records in data:
df_dog.count()

490

In [7]:
#To check the number of the columns in data:
df_dog.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [9]:
#To print the schema:
df_dog.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [12]:
#To print the five point summary:
df_dog.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [14]:
from pyspark.ml.feature import VectorAssembler


In [16]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [18]:
output = assembler.transform(df_dog)

In [21]:
final_data = output.select('features','Spoiled')
final_data.show()

+-------------------+-------+
|           features|Spoiled|
+-------------------+-------+
| [4.0,2.0,12.0,3.0]|    1.0|
| [5.0,6.0,12.0,7.0]|    1.0|
| [6.0,2.0,13.0,6.0]|    1.0|
| [4.0,2.0,12.0,1.0]|    1.0|
| [4.0,2.0,12.0,3.0]|    1.0|
|[10.0,3.0,13.0,9.0]|    1.0|
| [8.0,5.0,14.0,5.0]|    1.0|
| [5.0,8.0,12.0,8.0]|    1.0|
| [6.0,5.0,12.0,9.0]|    1.0|
| [3.0,3.0,12.0,1.0]|    1.0|
| [9.0,8.0,11.0,3.0]|    1.0|
|[1.0,10.0,12.0,3.0]|    1.0|
|[1.0,5.0,13.0,10.0]|    1.0|
|[2.0,10.0,12.0,6.0]|    1.0|
|[1.0,10.0,11.0,4.0]|    1.0|
| [5.0,3.0,12.0,2.0]|    1.0|
| [4.0,9.0,11.0,8.0]|    1.0|
| [5.0,1.0,11.0,1.0]|    1.0|
|[4.0,9.0,12.0,10.0]|    1.0|
| [5.0,8.0,10.0,9.0]|    1.0|
+-------------------+-------+
only showing top 20 rows



## Train test split

In [22]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [25]:
#To check the number of records in train data:
train_data.count()

343

In [26]:
#To check the number of records in test data:
test_data.count()

147

In [27]:
#To print the five point summary for train data:
train_data.describe().show()

+-------+-------------------+
|summary|            Spoiled|
+-------+-------------------+
|  count|                343|
|   mean|0.31486880466472306|
| stddev| 0.4651421498634166|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [28]:
#To print the five point summary for test data:
test_data.describe().show()

+-------+-------------------+
|summary|            Spoiled|
+-------+-------------------+
|  count|                147|
|   mean|0.21768707482993196|
| stddev|  0.414084347220005|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [29]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

In [38]:
dtc = DecisionTreeClassifier(labelCol='Spoiled', featuresCol='features')
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')
gbtc = GBTClassifier(labelCol='Spoiled', featuresCol='features')

In [39]:
dtm = dtc.fit(train_data)
rfm = rfc.fit(train_data)
gbtm = gbtc.fit(train_data)

In [40]:
dtc_predictions = dtm.transform(test_data)
rfc_predictions = rfm.transform(test_data)
gbt_predictions = gbtm.transform(test_data)

In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [43]:
evall = MulticlassClassificationEvaluator(labelCol='Spoiled', predictionCol='prediction', metricName='accuracy')

In [44]:
dtc_acc = evall.evaluate(dtc_predictions)
rfc_acc = evall.evaluate(rfc_predictions)
gbt_acc = evall.evaluate(gbt_predictions)

In [46]:
print("Here are the results!")
print("\n")
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print("\n")
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print("\n")
print('A ensemble using GBT had an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!


A single decision tree had an accuracy of: 97.28%


A random forest ensemble had an accuracy of: 97.28%


A ensemble using GBT had an accuracy of: 97.28%


In [48]:
rfm.featureImportances

SparseVector(4, {0: 0.0265, 1: 0.0145, 2: 0.9369, 3: 0.0221})

In [49]:
#From the above results we can see that percentage of preservative c is causing the problem.
#Chemical C is the problem